<a href="https://colab.research.google.com/github/HyunMooKim/AI-Data/blob/main/KRX_Contest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read me

매매 방법은 다음과 같다.

매수: 앞에서 설명한 세가지 기술적 지표들을 종합적으로 고려하기 위해 각
값들의 가중치를 구했다. 이후 각각의 가중치들을 합하여 보유자금에서 가중치만
큼 주식을 매수한다.

추세추종: 널리 알려진 추제 주종에 따르면 PB(Bollinger band)값이 0.8보다
크고 MFI(Money flow index) 값이 80보다 큰 경우 매수한다. 다만, 가중치로 표현
하기 위해 PB와 MFI를 각각 0.8,80으로 나누었다.

PER 0.2는 PER의 값이 0.2 미만인 경우는 0의 값을 주고 0.2에 가까울수록
1에 가깝게, PER값이 커질수록 0에 가깝게 가중치를 부여한다.

3% 룰은 거래량이 10,000,000 이상이며 전날대비 가격 변동률이 0.03이상이
며 종가나 거래량이 0 아닌 경우의 주식들을 선정한다. 이후 개별주식/선정된 주
식들의 전체 거래량으로 가중치를 구했다.

각각 방식의 가중치 코드는 PER 0.2는 a_weight에서, 추세추종은 b_weight에
서 그리고 3%룰은 c_weight에서 확인 가능하다. 또 값이 클수록 매수 신호가 강
하다.

매도: 매수한 주식 중 보유한지 일정 기간(hold_days)이 지났거나 현재가격
이 구매가격 대비 low_percent만큼 낮은 경우 혹은 현재가격이 high_percent만큼
높은 경우 혹은 최근 가격 변동률이 spike_percent이상인 경우엔 매도 대상으로
선택한다.

# KRX contest 본선 최종 제출 코드

아래 코드를 whl 파일로 제출했다.


In [ ]:
import kquant as kq
import datetime as dt
from datetime import timedelta
import logging
import pandas as pd
import numpy as np
def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    r"""주식매매 지시함수

    주식매매 지시함수에 대한 설명

    :param dt.date date: 매매일 날짜
    :param dict[str, pd.DataFrame] dict_df_result: 매매일까지의 주문 및 체결 정보
    :param dict[str, pd.DataFrame] dict_df_position: 매매일의 현재 주식 보유 정보
    :param logging.Logger logger: 로거
    :return list[tuple[str, int]]: 주식매매 지시
    :return list[tuple[str, int]]: 주식매매 지시
    """


    # 현재 보유 현금 불러오기
    try:
        cash = dict_df_result['TOTAL']['CASH'].iloc[-1]
    except:
        cash = 1_000_000_000

    handed_keys = 0 # 가지고 있는 주식 종류 개수
    for i in dict_df_position.keys():
        if len(dict_df_position[i]) > 0:
            handed_keys += 1

    will_sell_symbol = [] # 오늘 매도할 주식 종목코드 후보
    real_sell_symbol = [] # 오늘 진짜 매도할 주식 종목코드(매수 후보랑 비교해서 안 겹치는 것들만)

    will_buy_symbol = [] # 오늘 매수할 주식 종목코드 후보
    real_buy_symbol = [] # 오늘 진짜 매수할 주식 종목코드(매도 후보랑 비교해서 안 겹치는 것들만)


    # 매수 매도 기준
    hold_days = 10 # 주식 가지고 있을 날짜
    low_percent = -0.1 # 구매가 대비 손절 기준
    drop_percent = -0.1 # 전날 대비 손절 기준
    high_percent = 0.12 # 구매가 대비 익절 기준
    spike_percent = 0.1 # 전날 대비 익절 기준
    min_stocks = 5 # min종목
    max_stocks = 16 # max종목
    marketcap_n = 600 # 시가 총액 하위 n개에서만 매수매도
    per = 0.2 # per 기준
    timid = 0.5 # 장이 안좋다고 판단될 경우 현금 일부 투자
    moving = 5 # 장을 판단하는 이동평균 기준


    #장이 안 좋을 경우 바뀌는 매수매도 기준. 현금 보유도 늘린다.
    if int(kq.daily_kosdaq_index('1', date, date)['CLOSE']) <= np.average(kq.daily_kosdaq_index('1', date - dt.timedelta(days = moving), date)['CLOSE'] ):
        drop_percent= -0.07
        low_percent = -0.07
        cash = cash * timid
        high_percent = 0.07
        spike_percent = 0.07
        max_stocks = 10

    symbols_and_orders = [] # 출력값
    try:
        kq.daily_kospi_index('1', date, date) # 장이 열린 날만 코드 진행

        # 매도 후보 선정
        for i in dict_df_result.keys():
            if i == 'TOTAL': # TOTAL은 무시
                continue
            try:
                if int(dict_df_result[i]['QTY'].sum()) > 0: # 보유중인 것만 매도 판단 대상
                    # 보유 일수가 지난 것
                    # 구매가보다 low_percent 낮은 것
                    # 구매가보다 high_percent 높은 것
                    # 어제 가격보다 drop_pervcent 떨어진 것
                    today_price = kq.daily_stock(i, date, date)['CLOSE'].iloc[-1]
                    percentage_change = int(dict_df_result[i]['PRICE'].pct_change().fillna(0).iloc[-1])
                    if (int(dict_df_result[i]['QTY'].iloc[-hold_days:].sum()) == 0) \
                    or ( today_price < dict_df_position[i]['TRADE_PRICE'].iloc[-1] * (1 + low_percent)) \
                    or ( today_price > dict_df_position[i]['TRADE_PRICE'].iloc[-1] * (1 + high_percent)) \
                    or (percentage_change <= drop_percent) \
                    or (percentage_change >= spike_percent) :
                        will_sell_symbol.append(i) # 매도 예정 리스트에 종목코드 추가
                        handed_keys -= 1

            except:
                pass

        f=0
        # 매수 후보 선정

        if (handed_keys <= min_stocks): # 가지고 있는(가지고 있을) 주식 종류 개수가 min_stocks 이하라면 주식 구매
            f=1

            # 매수 기준 가중치 생성시 필요한 정보를 모두 df에 담는다.
            df_symbols = kq.symbol_stock()
            all_stock_list = df_symbols['SYMBOL']
            start_date = kq.daily_kosdaq_index('1', date + timedelta(days = -30), date)['DATE'].iloc[-20] # start_date = 오늘로부터 20일전
            window_size = 20
            window = 10
            df = {}
            for symbol in all_stock_list.values:
                try:
                    stock_info = kq.daily_stock(symbol, start_date, date) #과거 데이터 20일치
                    stock_info = stock_info[['DATE', 'SYMBOL', 'CLOSE', 'HIGH','LOW', 'VOLUME', 'CHG_PCT','MARKETCAP','SHARES']]

                    # 볼린저 밴드 관련 column 생성
                    stock_info['ma20'] = stock_info['CLOSE'].rolling(window_size).mean() # 종가 20일 평균
                    stock_info['std'] = stock_info['CLOSE'].rolling(window_size).std()
                    stock_info['upper'] = stock_info['ma20'] + 2 * stock_info['std'] # 중간 볼린저 밴드 + (2 x 표준편차)
                    stock_info['lower'] = stock_info['ma20'] - 2 * stock_info['std'] # 중간 볼린저 밴드 - (2 x 표준편차)
                    stock_info['pb'] = (stock_info['CLOSE'] - stock_info['lower']) / ( (stock_info['upper'] - stock_info['lower']) + 1e-10)
                    stock_info['bandwidth'] = (stock_info['upper'] - stock_info['lower']) / (stock_info['ma20'] * 100  + 1e-10)
                    stock_info['TP'] = (stock_info['CLOSE']+stock_info['HIGH']+stock_info['LOW'])/3  #중심가격 typical price
                    stock_info['PMF'] = 0 # 긍정적 현금 흐름
                    stock_info['NMF'] = 0 # 부정적 현금 흐름

                    for i in range(len(stock_info.CLOSE)-1):
                        # 주가 상승 -> positive money flow = price * volume
                        if stock_info.TP.values[i] < stock_info.TP.values[i+1]:
                            stock_info.PMF.values[i+1] = stock_info.TP.values[i+1] * stock_info.VOLUME.values[i+1]
                        # 주가 하락 -> negative money flow = price * volume
                        else:
                            stock_info.NMF.values[i+1] = stock_info.TP.values[i+1] * stock_info.VOLUME.values[i+1]
                    #Money Flow Index
                    stock_info['MFI10'] = 100 - 100 / (1 + stock_info.PMF.rolling(window).sum() / stock_info.NMF.rolling(window).sum()  + 1e-10 )
                    stock_info = stock_info[19:] # NaN 자르기 - 필요한만큼 잘라야함
                    stock_info['PER'] = kq.info_stock(symbol)[['PER']] # info_stock에서 이미 계산된 PER 가져오기 -> 없는 값이 너어무 많음

                    df[symbol] = stock_info
                except:
                    pass


            df2 = pd.DataFrame()
            for symbol in df.keys():
                df2 = pd.concat([df2, df[symbol].query('DATE == @date')], axis = 0, ignore_index = True)
            df2.columns = df['900110'].columns # 데이터 프레임의 열에 이름 부여

            # 1. 필터링 시가총액 하위 marketcap_n 개
            df2 = df2.sort_values(by = ['MARKETCAP']).head(marketcap_n)
            df2['pb'].fillna(0, inplace = True) # 값 없는 것들은 0으로 만들기
            df2['MFI10'].fillna(0,inplace = True) # 값 없는 것들은 0으로 만들기

            # 2. PER 기준 신호 생성
            # df2['a_weight'] = np.where(df2['PER'] < per , 0, 1 )
            # PER 신호의 가중치 계산. PER이 0.2 미만인 경우 가중치 0으로 설정(너무 낮아서 부정적이라 판단). 그 이상인 경우 PER이 낮을수록(0.2에 가까울수록) 1에 가깝고 클수록 0에 가깝게 역수 취해줌.
            df2['a_weight'] = np.where(df2['PER'] >= per , ( per / df2['PER']) , 0 )

            # 3. 추세추종 기준 신호 생성
            df2['b_weight'] = df2['pb']/0.8 + df2['MFI10']/80 # 값이 클수록 매수

            # 4. 3%룰 신호 생성
            # 3%룰 만족하는 종목들만 거래량 남김. 나머지는 0
            df2['c_weight'] = np.where((df2['VOLUME'] > 10_000_000) & (df2['CHG_PCT'] >= 0.03) & (df2['CLOSE'] != 0), df2['VOLUME'], 0)
            df2['c_weight'] =  df2['c_weight'] / df2['c_weight'].sum() # 남은 거래량을 정규화

            # 가중치 합산
            df2['total_weight'] = df2['a_weight'] + df2['b_weight'] +  df2['c_weight']
            # 가중치가 높은 상위 max_stocks 내림차순
            df_all = df2[['SYMBOL', 'CLOSE', 'total_weight']].sort_values(by = ['total_weight'], ascending = False).head(max_stocks - handed_keys)
            df_all.reset_index(drop=True, inplace=True)

            # 주식 매입
            for i in range(0, len(df_all)):
                will_buy_symbol.append(df_all['SYMBOL'].iloc[i])

            # 중복 제거(후보들 중에서 찐으로 사고 팔 것들만 리스트 다시)
            for i in range(0, len(will_sell_symbol)): # 판매 장바구니에서 구매 장바구니에 없는 것만 판매
                if will_sell_symbol[i] not in will_buy_symbol:
                    real_sell_symbol.append(will_sell_symbol[i])

            for i in range(0, len(will_buy_symbol)): # 구매 장바구니에서 판매 장바구니에 없는 것만 구매
                if will_buy_symbol[i] not in will_sell_symbol:
                    real_buy_symbol.append(will_buy_symbol[i])
                else:
                    df_all.drop(index = i, inplace = True)

            # 매수
            try:
                for i in range(0, len(df_all)):
                    # 각 주식 당 살 수 있는 금액. 종류별 주식을 가중치를 부여하여 금액 산정
                    each_cash = cash * (df_all['total_weight'].iloc[i] / df_all['total_weight'].sum())
                    after_each_cash = (each_cash / 1.001)

                    if after_each_cash > 1001:
                        buy_number = after_each_cash // df_all['CLOSE'].iloc[i]

                        if buy_number > 0:
                            symbols_and_orders.append((real_buy_symbol[i], buy_number))

            except:
                pass

        # 매도
        if f==0: # 매수 안한 날 -> 중복 체크 안된 리스트에서 매도
            for i in will_sell_symbol:
                today_price = kq.daily_stock(i, date, date)['CLOSE'].iloc[-1]
                before_tax = today_price * int(dict_df_result[i]['POSITION'].iloc[-1])
                if before_tax * 0.002 < 1: # 수수료 1원 미만일 경우 1원으로 올림
                    after_tax = before_tax - 2
                elif before_tax * 0.001 < 1:
                    after_tax = before_tax * 0.998 - 1
                else:
                    after_tax = before_tax * 0.997
                cash += after_tax
                symbols_and_orders.append((i, int(dict_df_result[i]['POSITION'].iloc[-1]) * (-1)))
        if f==1: # 매수 한 날 -> 중복 체크 된 리스트에서 매도
            for i in real_sell_symbol:
                today_price = kq.daily_stock(i, date, date)['CLOSE'].iloc[-1]
                before_tax = today_price * int(dict_df_result[i]['POSITION'].iloc[-1])
                if before_tax * 0.002 < 1: # 수수료 1원 미만일 경우 1원으로 올림
                    after_tax = before_tax - 2
                elif before_tax * 0.001 < 1:
                    after_tax = before_tax * 0.998 - 1
                else:
                    after_tax = before_tax * 0.997
                cash += after_tax
                symbols_and_orders.append((i, int(dict_df_result[i]['POSITION'].iloc[-1]) * (-1)))

    except Exception as e:
        print('장이 안열리는 사태가 발생.')
        print(f"An error occurred: {e}")

    return symbols_and_orders


# 연습 코드

효율성을 위해 미리 매매에 필요한 데이터를 대회측에서 제공한 Kquant 패키지로 불러와 DataFrame으로 구성하고 이를 이용해 백 테스트를 진행했습니다.

2023 1월 ~ 2023 9월 데이터로 백테스트를 해보고 가장 좋았던 모델을 제출했습니다.

아래 코드는 실제 제출 코드 결과는 아니지만, 백테스트를 하던 코드입니다.

In [ ]:
import kquant as kq
import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import logging
import pandas as pd
import numpy as np
import pickle

In [ ]:
with open('2023주식.pickle', 'rb') as f:
    df = pickle.load(f)

df['900110']

,DATE,SYMBOL,CLOSE,HIGH,LOW,VOLUME,CHG_PCT,MARKETCAP,SHARES,dt_VOLUME,PER,ma20,std,upper,lower,pb,bandwidth,TP,PMF,NMF,MFI10
0,2023-01-02,900110,179,181,173,"2,060,035",2.8700,"52,255,836,950","291,932,050",0.7471,18.0105,NaN,NaN,NaN,NaN,NaN,NaN,177.6667,0,0,NaN
1,2023-01-03,900110,171,180,169,"3,599,183",-4.4700,"49,920,380,550","291,932,050",0.7471,17.2056,NaN,NaN,NaN,NaN,NaN,NaN,173.3333,0,"623,858,386",NaN
2,2023-01-04,900110,170,174,169,"2,817,903",-0.5800,"49,628,448,500","291,932,050",-0.2171,17.1050,NaN,NaN,NaN,NaN,NaN,NaN,171.0000,0,"481,861,413",NaN
3,2023-01-05,900110,171,173,170,"1,684,473",0.5900,"49,920,380,550","291,932,050",-0.4022,17.2056,NaN,NaN,NaN,NaN,NaN,NaN,171.3333,"288,606,374",0,NaN
4,2023-01-06,900110,171,172,168,"1,652,477",0.0000,"49,920,380,550","291,932,050",-0.0190,17.2056,NaN,NaN,NaN,NaN,NaN,NaN,170.3333,0,"281,471,915",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2023-09-21,900110,135,137,133,"2,096,325",-1.4600,"39,410,826,750","291,932,050",0.3791,13.5834,139.5000,2.5854,144.6708,134.3292,0.0649,7.4133,135.0000,0,"283,003,875",17.4185
181,2023-09-22,900110,133,135,132,"1,370,951",-1.4800,"38,826,962,650","291,932,050",-0.3460,13.3821,139.3500,2.8704,145.0909,133.6091,-0.0530,8.2395,133.3333,0,"182,793,466",17.4318
182,2023-09-25,900110,132,138,131,"3,243,240",-0.7500,"38,535,030,600","291,932,050",1.3657,13.2815,139.0500,3.3003,145.6506,132.4494,-0.0340,9.4939,133.6667,"433,513,079",0,20.9417
183,2023-09-26,900110,130,133,129,"1,862,568",-1.5200,"37,951,166,500","291,932,050",-0.4257,13.0803,138.4500,3.7902,146.0304,130.8696,-0.0574,10.9504,130.6667,0,"243,375,551",22.0943


In [ ]:

# 한 달 백테스트 함수

def backtest_1months(start_date):

    init_cash = 1_000_000_000 # 초기 자금
    broker_fee_percent = 0.1 # 증권사 수수료
    trade_tax_percent = 0.2 # 매도시 주식양도세

    end_date = (start_date + relativedelta(months = 1) + timedelta(days = -1)) # 시작 날짜로부터 한 달 후
    date_list = pd.date_range(start=start_date, end=end_date, freq='D') # 시작 날짜와 종료 날짜 사이의 날짜들을 1일 간격 리스트로 만들기

    dict_df_result, dict_df_position = kq.backtest_update_stock_port_daily([], start_date, init_cash = init_cash) # 초기 dict_df_result, dict_df_position 선언. 처음에는 비어있음

    for i in date_list:

        i = i.date()

        try: # 오늘 장 여는지 안 여는지
            kq.daily_kospi_index('1', i, i)
        except ValueError:
            continue

        # 사고팔게 있음 사고팔아 포트폴리오 업데이트
        dict_df_result, dict_df_position = kq.backtest_update_stock_port_daily(
                    trade_func(i,dict_df_result, dict_df_position,logger = None), i, dict_df_result, dict_df_position, init_cash = init_cash, broker_fee_percent = broker_fee_percent, trade_tax_percent = trade_tax_percent)

    # 사프지수 계산
    init_cash=1_000_000_000
    value = {}
    value['투자기간의 매매일수'] = len(dict_df_result['TOTAL']['TOTAL_VALUE'])
    value['초기 투자금액'] = init_cash
    value['n번째 매매일의 일간 수익률'] = []
    value['일간 수익률 변동성'] = 0

    for i in range(1, value['투자기간의 매매일수']):
        if i == 1:
            value['n번째 매매일의 일간 수익률'].append(((dict_df_result['TOTAL']['TOTAL_VALUE'].iloc[i] / init_cash) - 1)*100)

        else:
            value['n번째 매매일의 일간 수익률'].append(((dict_df_result['TOTAL']['TOTAL_VALUE'].iloc[i] / dict_df_result['TOTAL']['TOTAL_VALUE'].iloc[i - 1]) - 1) *100)

    value['일간 수익률의 평균'] = np.mean(value['n번째 매매일의 일간 수익률'])

    for i in range(0, value['투자기간의 매매일수'] - 1):
        value['일간 수익률 변동성'] += ((value['n번째 매매일의 일간 수익률'][i] - value['일간 수익률의 평균']) ** 2) / (value['투자기간의 매매일수'] - 2)

    value['일간 수익률 변동성'] = value['일간 수익률 변동성'] ** 0.5
    value['평균 일간 수익률'] = ((dict_df_result['TOTAL']['TOTAL_VALUE'].iloc[-1] / init_cash) - 1) / (value['투자기간의 매매일수'] - 1) * 100
    value['샤프지수'] = (value['평균 일간 수익률'] / value['일간 수익률 변동성']) * (252 **0.5)

    finalscore = (value['샤프지수'] + value['평균 일간 수익률']) / 2

    print('{}월 샤프지수 : '.format(start_date.month),value['샤프지수'])
    print('{}월 수익률 : '.format(start_date.month), (dict_df_result['TOTAL']['TOTAL_VALUE'].iloc[-1] / init_cash) -1)
    print('{}월 최종 점수 : '.format(start_date.month), finalscore)

    return dict_df_result, dict_df_position , value['샤프지수'], value['평균 일간 수익률']
print('gogo')

gogo


In [ ]:
import time
start_time = time.time()
print('go')
final_list = []
profit_list = []
for i in range(1,10):
    a,b,c,d = backtest_1months(dt.date(2023,i,1))
    final = (c+d) / 2
    final_list.append( final )
    profit_list.append( d )

end_time = time.time()
elapsed_time = end_time - start_time
elapsed_time_minutes = elapsed_time / 60
print(f"Elapsed time: {elapsed_time_minutes:.2f} minutes")
print('평균 일간 수익률 1~9: ', profit_list)
print('최종 점수 1~9: ', final_list)

go


[2023-01-02] 종목: 331520, 주문전 보유수량:      0 주문수량: 53,270, 매매수량: 53,270, 주문후 보유수량: 53,270
[2023-01-02] 종목: 019570, 주문전 보유수량:      0 주문수량: 35,319, 매매수량: 35,319, 주문후 보유수량: 35,319
[2023-01-02] 종목: 032980, 주문전 보유수량:      0 주문수량: 27,090, 매매수량: 27,090, 주문후 보유수량: 27,090
[2023-01-02] 종목: 140520, 주문전 보유수량:      0 주문수량: 12,505, 매매수량: 12,505, 주문후 보유수량: 12,505
[2023-01-02] 종목: 025560, 주문전 보유수량:      0 주문수량: 16,585, 매매수량: 16,585, 주문후 보유수량: 16,585
[2023-01-02] 종목: 000950, 주문전 보유수량:      0 주문수량:    661, 매매수량:    661, 주문후 보유수량:    661
[2023-01-02] 종목: 024800, 주문전 보유수량:      0 주문수량: 10,369, 매매수량: 10,369, 주문후 보유수량: 10,369
[2023-01-02] 종목: 040610, 주문전 보유수량:      0 주문수량: 15,147, 매매수량: 15,147, 주문후 보유수량: 15,147
[2023-01-02] 종목: 016250, 주문전 보유수량:      0 주문수량:    773, 매매수량:    773, 주문후 보유수량:    773
[2023-01-02] 종목: 006200, 주문전 보유수량:      0 주문수량: 20,821, 매매수량: 20,821, 주문후 보유수량: 20,821
[2023-01-02] 종목: 094970, 주문전 보유수량:      0 주문수량:  5,928, 매매수량:  5,928, 주문후 보유수량:  5,928
[2023-01-02] 종목: 000850, 주문전 보유수량:      0 주

1월 샤프지수 :  8.818456017322369
1월 수익률 :  0.05388910299999994
1월 최종 점수 :  4.551041437608553


[2023-02-01] 종목: 007120, 주문전 보유수량:      0 주문수량: 23,629, 매매수량: 23,629, 주문후 보유수량: 23,629
[2023-02-01] 종목: 024850, 주문전 보유수량:      0 주문수량: 45,906, 매매수량: 45,906, 주문후 보유수량: 45,906
[2023-02-01] 종목: 038530, 주문전 보유수량:      0 주문수량: 73,697, 매매수량: 73,697, 주문후 보유수량: 73,697
[2023-02-01] 종목: 094940, 주문전 보유수량:      0 주문수량:  5,427, 매매수량:  5,427, 주문후 보유수량:  5,427
[2023-02-01] 종목: 006920, 주문전 보유수량:      0 주문수량: 11,966, 매매수량: 11,966, 주문후 보유수량: 11,966
[2023-02-01] 종목: 073110, 주문전 보유수량:      0 주문수량:  6,759, 매매수량:  6,759, 주문후 보유수량:  6,759
[2023-02-01] 종목: 044480, 주문전 보유수량:      0 주문수량: 28,023, 매매수량: 28,023, 주문후 보유수량: 28,023
[2023-02-01] 종목: 154030, 주문전 보유수량:      0 주문수량: 12,219, 매매수량: 12,219, 주문후 보유수량: 12,219
[2023-02-01] 종목: 206400, 주문전 보유수량:      0 주문수량: 16,326, 매매수량: 16,326, 주문후 보유수량: 16,326
[2023-02-01] 종목: 049520, 주문전 보유수량:      0 주문수량: 12,504, 매매수량: 12,504, 주문후 보유수량: 12,504
[2023-02-01] 종목: 043360, 주문전 보유수량:      0 주문수량: 13,978, 매매수량: 13,978, 주문후 보유수량: 13,978
[2023-02-01] 종목: 187660, 주문전 보유수량:      0 주

2월 샤프지수 :  -2.275959529144371
2월 수익률 :  -0.020526664000000028
2월 최종 점수 :  -1.1919973014142908


...

7월 샤프지수 :  -5.411573800049005
7월 수익률 :  -0.06436034499999999
7월 최종 점수 :  -2.8666877625245024


[2023-08-02] 종목: 021050, 주문전 보유수량:      0 주문수량: 26,768, 매매수량: 26,768, 주문후 보유수량: 26,768
[2023-08-02] 종목: 290740, 주문전 보유수량:      0 주문수량:  3,762, 매매수량:  3,762, 주문후 보유수량:  3,762
[2023-08-02] 종목: 054180, 주문전 보유수량:      0 주문수량: 13,575, 매매수량: 13,575, 주문후 보유수량: 13,575
[2023-08-02] 종목: 204210, 주문전 보유수량:      0 주문수량:  6,558, 매매수량:  6,558, 주문후 보유수량:  6,558
[2023-08-02] 종목: 002800, 주문전 보유수량:      0 주문수량:  5,167, 매매수량:  5,167, 주문후 보유수량:  5,167
[2023-08-02] 종목: 115610, 주문전 보유수량:      0 주문수량:  8,563, 매매수량:  8,563, 주문후 보유수량:  8,563
[2023-08-02] 종목: 066910, 주문전 보유수량:      0 주문수량: 12,681, 매매수량: 12,681, 주문후 보유수량: 12,681
[2023-08-02] 종목: 229000, 주문전 보유수량:      0 주문수량:  6,150, 매매수량:  6,150, 주문후 보유수량:  6,150
[2023-08-02] 종목: 072990, 주문전 보유수량:      0 주문수량:  3,065, 매매수량:  3,065, 주문후 보유수량:  3,065
[2023-08-02] 종목: 376930, 주문전 보유수량:      0 주문수량: 11,141, 매매수량: 11,141, 주문후 보유수량: 11,141
[2023-08-02] 종목: 131090, 주문전 보유수량:      0 주문수량: 21,826, 매매수량: 21,826, 주문후 보유수량: 21,826
[2023-08-02] 종목: 286750, 주문전 보유수량:      0 주

8월 샤프지수 :  4.814334350391405
8월 수익률 :  0.03287430000000002
8월 최종 점수 :  2.4893529251957025


[2023-09-01] 종목: 264850, 주문전 보유수량:      0 주문수량: 11,668, 매매수량: 11,668, 주문후 보유수량: 11,668
[2023-09-01] 종목: 013720, 주문전 보유수량:      0 주문수량: 15,240, 매매수량: 15,240, 주문후 보유수량: 15,240
[2023-09-01] 종목: 322180, 주문전 보유수량:      0 주문수량:  2,175, 매매수량:  2,175, 주문후 보유수량:  2,175
[2023-09-01] 종목: 376930, 주문전 보유수량:      0 주문수량:  7,922, 매매수량:  7,922, 주문후 보유수량:  7,922
[2023-09-01] 종목: 014990, 주문전 보유수량:      0 주문수량: 25,693, 매매수량: 25,693, 주문후 보유수량: 25,693
[2023-09-01] 종목: 098120, 주문전 보유수량:      0 주문수량:  2,820, 매매수량:  2,820, 주문후 보유수량:  2,820
[2023-09-01] 종목: 142210, 주문전 보유수량:      0 주문수량:  5,786, 매매수량:  5,786, 주문후 보유수량:  5,786
[2023-09-01] 종목: 032750, 주문전 보유수량:      0 주문수량:  3,856, 매매수량:  3,856, 주문후 보유수량:  3,856
[2023-09-01] 종목: 219420, 주문전 보유수량:      0 주문수량:  4,000, 매매수량:  4,000, 주문후 보유수량:  4,000
[2023-09-01] 종목: 057030, 주문전 보유수량:      0 주문수량:  6,101, 매매수량:  6,101, 주문후 보유수량:  6,101
[2023-09-01] 종목: 127120, 주문전 보유수량:      0 주문수량:  6,243, 매매수량:  6,243, 주문후 보유수량:  6,243
[2023-09-01] 종목: 002920, 주문전 보유수량:      0 주

9월 샤프지수 :  0.5683494745193878
9월 수익률 :  0.003004487999999972
9월 최종 점수 :  0.2925205372596938
Elapsed time: 26.00 minutes
평균 일간 수익률 1~9:  [0.2836268578947365, -0.10803507368421067, 0.15822177142857147, 0.11915046842105287, 0.15617152631579007, 0.22515311000000038, -0.3218017249999999, 0.16437150000000011, 0.01669159999999984]
최종 점수 1~9:  [4.551041437608553, -1.1919973014142908, 1.3140408680199813, 0.9629151097120617, 1.3358928839505702, 2.0185184100673603, -2.8666877625245024, 2.4893529251957025, 0.2925205372596938]
